<a href="https://colab.research.google.com/github/zhengwu123/Machine_learning_group_project/blob/master/spam%20confusion%20matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1. (k-NN classifier and 10-fold cross validation)
We want to train a k-NN classifier for the Iris Data Set. Use 10-fold cross validation to select a good
k from [1,50] for the k-NN classifier. Submit k and the source code

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import sys
import time
import numpy as np
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from pydotplus import graph_from_dot_data
from sklearn.svm import SVC
import pandas as pd
from sklearn.impute import SimpleImputer
# get iris data
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)

In [ ]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors=5)
#set fold to 10
scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#find best k
#reference:
#https://www.ritchieng.com/machine-learning-cross-validation/
k_range = range(1, 50)
# empty list to store scores
k_scores = []

# loop throguh 1 to 50 to find optimal k
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())


print(k_scores)
print('Max of list', max(k_scores))

# plot how accuracy changes as we vary k


# plot the value of K for KNN (x-axis) versus the cross-validated accuracy (y-axis)
# plt.plot(x_axis, y_axis)
x_ticks = []
for i in range(1,50,2):
  x_ticks.append(i);
plt.plot(k_range, k_scores)
plt.xticks(x_ticks)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-validated accuracy')


Convert the problem in Iris Data Set into a binary classification task (setosa versus non-setosa). We
can do so by replacing the class labels of the instances to non-setosa except for those that belong
to the setosa class.
Create a training set that contains 80% of the labeled data and create a test set that contains the
remaining 20%. Train a logistic regression classifier using the training set. Plot the ROC curve for
the setosa class (positive class) when applying the logistic regression classifier to the test set.
Submit the plot and the source code.

In [ ]:
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

iris = load_iris()
X = iris.data
y = iris.target
#print(X)
##print(y)

#preprocess label
for i in range(0,len(y) - 1):
  if y[i] != 0:
    y[i] = 1


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#train our logistic model
lg_classifier = LogisticRegression(random_state=0).fit(X_train, y_train)


start_time = time.time()
y_pred = lg_classifier.predict(X_test)

duration = (time.time() - start_time) * 1000
# confidence score and running time:
print("Knn running time milliseconds: %.2f" % duration)
error = (y_test != y_pred).sum()
accuracy = (1 - error / y_test.shape[0]) * 100
print('Misclassified samples: {}.'.format(error))
print('train accuracy: %.2f' % lg_classifier.score(X_train, y_train))
print("The Test accuracy is {:1.2f}%.".format(accuracy))


ns_probs = [0 for _ in range(len(y_test))]
# Compute ROC curve and ROC area for each class
lr_probs = lg_classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()
exit(0)

[Bonus question]

spam or not spam using random forest and SVM

The dataset has a total of 4,827 SMS legitimate messages (86.6%) and a total of 747 (13.4%)
spam messages. Please refer to https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection for
more detailed descriptions.

In [ ]:
# load data


In [ ]:


from google.colab import files
uploaded = files.upload()



In [ ]:
!unzip smsspamcollection.zip

In [ ]:
import pandas as pd
df = pd.read_table('SMSSpamCollection',
                   sep='\t',
                   header=None,
                   names=['label', 'sms_message'])
df['label'] = df.label.map({'ham':0, 'spam':1})
#print(df.head)

X = df['sms_message']
y = df['label']
#replace example@gmail.com with email address
X = X.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')
#replace urls
X = X.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Remove punctuation
X = X.str.replace(r'[^\w\d\s]', ' ')

# change words to lower case
X = X.str.lower()

print(X.head)
print(y.head)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
#generate features
from nltk.tokenize import word_tokenize

# create bag-of-words
all_words = []

for message in X:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)
print('Total Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))
#use top 2000 words as features
word_features = list(all_words.keys())[:2000]

In [ ]:
import numpy as np

# The find_features function will determine which of the 2000 word features are contained in the review

def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

#features for all messages
messages = list(zip(X, y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]
npfeatures = np.array(featuresets)
print(npfeatures[:,0])
#print(npfeatures[:,1])

In [ ]:
#1 using SVM
import time
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from nltk.classify.scikitlearn import SklearnClassifier

training, testing = train_test_split(featuresets, test_size = 0.2, random_state=seed)
#nptraining = np.array(training)
#print(nptraining)
svcModel = SklearnClassifier(SVC(kernel = 'linear'))

start_time = time.time()
svcModel.train(training)
from sklearn.metrics import confusion_matrix
#labels = svcModel.labels() #[0,1]
#print(labels)
# confidence score and running time:
duration = (time.time() - start_time) * 1000
print("svm running time milliseconds: %.2f" % duration)

#get predict_y labels
txt_features, y_labels = zip(*testing)
y_predict = svcModel.classify_many(txt_features)
# print accuracy
accuracy = nltk.classify.accuracy(svcModel, testing)*100
print("SVC Accuracy: {}".format(accuracy))
# make class label prediction for testing set

cf_matrix = confusion_matrix(y_labels, y_predict, labels=[0, 1])



In [ ]:
# plot confusion matrix for SVM
import seaborn as sns
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

In [ ]:
#2 create random forest classifier
import time
from sklearn.ensemble import RandomForestClassifier
from nltk.classify.scikitlearn import SklearnClassifier

rfModel = SklearnClassifier(RandomForestClassifier())

start_time = time.time()
rfModel.train(training)

#get predict_y labels
txt_features, y_labels_rf = zip(*testing)
y_predict_rf = rfModel.classify_many(txt_features)
# print accuracy

# confidence score and running time:
duration = (time.time() - start_time) * 1000
accuracy = nltk.classify.accuracy(rfModel, testing)*100
cf_matrix_rf = confusion_matrix(y_labels_rf, y_predict_rf, labels=[0, 1])
print("Random Forest Accuracy: {}".format(accuracy))


In [ ]:
# draw confusion matrix for random forest

import seaborn as sns
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix_rf, annot=labels, fmt='', cmap='Blues')